<a href="https://colab.research.google.com/github/eatcosmos/mcp-frp/blob/master/mcp-frp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 挂载网盘
/content/drive/MyDrive/code/ 用于存放代码

In [ ]:
import os
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

# 下载代码

In [ ]:
# 如果 /content/drive/MyDrive/code/mcp-frp/ 不存在则下载 https://github.com/eatcosmos/mcp-frp
!mkdir -p /content/drive/MyDrive/code/
!rm -rf /content/drive/MyDrive/code/mcp-frp/
if not os.path.exists('/content/drive/MyDrive/code/mcp-frp/'):
  %cd /content/drive/MyDrive/code/
  !git clone https://github.com/eatcosmos/mcp-frp.git

# 准备工作！
1. 准备一个新账号，淘宝充值Colab PRO
2. 把自己电脑常用的 id_rsa.pub 文件替换 /content/drive/MyDrive/code/mcp-frp/id_rsa.pub

In [ ]:

!chmod +x /content/drive/MyDrive/code/mcp-frp/setupSSH.sh
#PRO版本在终端里面运行 /content/drive/MyDrive/code/mcp-frp/setupSSH.sh
!/content/drive/MyDrive/code/mcp-frp/setupSSH.sh
# !ssh -p 22 root@127.0.0.1

In [ ]:
from google.colab import userdata

%cd /content/drive/MyDrive/code/mcp-frp/
!cp ./frp_0.63.0_linux_amd64/frpc-template.toml ./frp_0.63.0_linux_amd64/frpc.toml
# 替换 ./frp_0.63.0_linux_amd64/frpc.toml 里面 serverAddr = "127.0.0.1" 为 serverAddr = userdata.get('FRPS_SERVER_IP')
# 获取FRPS服务器IP
frps_ip = userdata.get('FRPS_SERVER_IP')

# 定义文件路径
file_path = './frp_0.63.0_linux_amd64/frpc.toml'

# 读取配置文件内容
with open(file_path, 'r') as file:
    content = file.read()

# 替换服务器地址
new_content = content.replace(
    'serverAddr = "127.0.0.1"',
    f'serverAddr = "{frps_ip}"'
)

# 写回配置文件
with open(file_path, 'w') as file:
    file.write(new_content)

print("配置文件更新完成！")

In [ ]:
if not os.path.exists('/content/drive/MyDrive/code/.cursor/'):
  !mkdir -p /content/drive/MyDrive/code/.cursor
if not os.path.exists('/content/drive/MyDrive/code/.cursor-server/'):
  !mkdir -p /content/drive/MyDrive/code/.cursor-server
!rm -rf /root/.cursor
!ln -sf /content/drive/MyDrive/code/.cursor /root/
!rm -rf /root/.cursor-server
!ln -sf /content/drive/MyDrive/code/.cursor-server /root/
#
!ls -la /root/.cursor-server

In [ ]:
%cd /content/drive/MyDrive/code/mcp-frp/frp_0.63.0_linux_amd64/
!chmod +x ./frpc
# !cat ./frpc.toml
!./frpc -c ./frpc.toml

/content/drive/MyDrive/code/mcp-frp/frp_0.63.0_linux_amd64


如果无法访问可以在自己的笔记本上运行下
ssh -p 7502 root@FRPS_SERVER_IP